In [1]:
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
from dateparser import parse
import pandas as pd

import sqlalchemy
import mysql.connector
from mysql.connector import Error

In [3]:
user = 'umam'
password = 'AKuPastiBisa1'
host = 'localhost'
dbname = 'scrap'
port = 3306
conn = sqlalchemy.create_engine('mysql://{0}:{1}@{2}:{3}/{4}'.format(user, password, host, port, dbname))
try:
    create_table_jobstreet = """CREATE TABLE IF NOT EXISTS jobstreet ( 
                             job_id int(11) NOT NULL,
                             url_logo varchar(250),
                             company_name varchar(250),
                             location varchar(250),
                             position varchar(250),
                             industry varchar(250),
                             min_company_size int(11),
                             max_company_size int(11),
                             job_desc varchar(250),
                             posted DATE NOT NULL,
                             PRIMARY KEY (job_id)) """

    conn.execute(create_table_jobstreet) #create db
except:
    pass
#     conn = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
#                                                format(user, password, host, dbname))


In [41]:
def update_data_db(df):
    pass
    
def process_company_size(company_size):
    size = []
    if company_size != None:
        for token in company_size.split():
            if token.isdigit():
                size.append(int(token))
            else:
                pass
            
    if len(size) == 1:
        # add max company size if greater than 5000
        size.append(99999)
    elif len(size) == 0:
        size = [0,0]
    
    return size
        

def job_page(position, job_id):
    # visit job details webpage
    route = position.lower().replace(" ","-")+"-"+str(job_id)
    url = 'https://www.jobstreet.co.id/id/job/{}'.format(route)
    job_webpage = requests.get(url, headers=headers, params=pay_load)
    job_soup = BeautifulSoup(job_webpage.text, 'html')
    company_size = job_soup.find('p',{'id':'company_size'}).text if job_soup.find('p',{'id':'company_size'}) else None
    # process to min and max size
    min_size, max_size = process_company_size(company_size)
    # we will cleaning job desc ion preprocessing
    job_desc = job_soup.find('div',{'id':'job_description'}).text if job_soup.find('div',{'id':'job_description'}) else None
    # split simpler than regex, dont make it complex
    date_posted_str = job_soup.find('p', {'id':'posting_date'}).text.split(': ')[-1] if job_soup.find('p', {'id':'posting_date'}) else None
    # format into datetime format
    try:
        date_posted = parse(date_posted_str).strftime('%d/%m/%y')
    except TypeError:
        date_posted = parse("1/January/1990").strftime('%d/%m/%y')
        
    return min_size, max_size, job_desc, date_posted
    
def get_all_data(soup):
    # initiate all list data
    urls_logo, companies_name, locations, positions, industries, jobs_id, min_companies_size, max_companies_size, jobs_desc, dates_posted = [[] for i in range(10)]
    
    for detail_job in soup.find_all('div',{'class':'panel-body','id':re.compile(r'job_ad_(\d+)$')}):
        urls_logo.append(detail_job.find('img', {'class':"img-company-logo"})["data-original"] if detail_job.find('img', {'class':"img-company-logo"}) else None)
        companies_name.append(detail_job.find('a', {'class':"company-name"}).span.string if detail_job.find('a', {'class':"company-name"}) else None)
        locations.append(detail_job.find('li', {'class':"job-location"})["title"] if detail_job.find('li', {'class':"job-location"}) else None)
        position = detail_job.find('a',{'class':'position-title-link'})["data-job-title"] if detail_job.find('a',{'class':'position-title-link'}) else None
        positions.append(position)
        
        industry_title = detail_job.find('a',{'class':'text-muted'})
        if industry_title:
            industries.append(re.search(r'Lowongan (.*?) di', industry_title["title"]).group(1))
        else:
            industries.append(None)
        job_id = detail_job.find('a',{'class':'position-title-link','data-job-id':re.compile(r'.*')})["data-job-id"] if detail_job.find('a',{'class':'position-title-link','data-job-id':re.compile(r'.*')}) else None
        jobs_id.append(job_id)
        
        min_size, max_size, job_desc, date_posted = job_page(position, job_id)
        min_companies_size.append(min_size)
        max_companies_size.append(max_size)
        jobs_desc.append(job_desc)
        dates_posted.append(date_posted)
        
    return urls_logo, companies_name, locations, positions, industries, jobs_id, min_companies_size, max_companies_size, jobs_desc, dates_posted

def main():
    URL = "https://www.jobstreet.co.id/id/job-search/job-vacancy.php?ojs=6"
    avail = True
    s = requests.session()
    page = 1
    urls_logo_all, companies_name_all, locations_all, positions_all, industries_all, jobs_id_all, min_companies_size_all, max_companies_size_all, jobs_desc_all, date_posted_all = [[] for i in range(10)]
    for i in range(1, 50):
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'}
        pay_load = {'key':'','area':1,'option':1,'pg':None,'classified':1,'src':16,'srcr':12}
        pay_load['pg'] = page
        webpage = requests.get(URL, headers=headers, params=pay_load)
        soup = BeautifulSoup(webpage.text,'html.parser')
        links = soup.find_all('a',{'class':'position-title-link','data-job-id':re.compile(r'.*')})
        if not len(links):
            break
        else:
            urls_logo, companies_name, locations, positions, industries, jobs_id, min_companies_size, max_companies_size, jobs_desc, date_posted = get_all_data(soup)
            urls_logo_all.extend(urls_logo)
            companies_name_all.extend(companies_name)
            locations_all.extend(locations)
            positions_all.extend(positions)
            industries_all.extend(industries)
            jobs_id_all.extend(jobs_id)
            min_companies_size_all.extend(min_companies_size)
            max_companies_size_all.extend(max_companies_size)
            jobs_desc_all.extend(jobs_desc)
            date_posted_all.extend(date_posted)

            page += 1
            
    df = pd.DataFrame.from_dict({"job_id":jobs_id_all,
                                "url_logo":urls_logo_all,
                                "company_name":companies_name_all,
                                "location":locations_all,
                                "position":positions_all,
                                "industry":industries_all,
                                "min_company_size":min_companies_size_all,
                                "max_company_size":max_companies_size_all,
                                "job_desc":jobs_desc_all,
                                "posted":date_posted_all})
        
    return df

In [42]:
df = main()

In [3]:
df = pd.read_csv("examples.csv")

In [4]:
df.head()

,Unnamed: 0,job_id,url_logo,company_name,location,position,industry,min_company_size,max_company_size,job_desc,posted
0,0,3068761,NaN,PT Glassindo Utama,Jakarta Barat (Jakarta Raya),Estimator & Purchasing,Pembelian/Manajemen Material,51,200,Requirement : \n \n\nUmur 25 th s/d 40 th\nMi...,23/09/19
1,1,3068762,https://siva.jsstatic.com/id/28632/images/logo...,PT Home Credit Indonesia,Banten - Tangerang Area,Field Collection - Tangerang,Perbankan/Keuangan,1001,2000,Job Description:Mengunjungi Customer sesuai d...,23/09/19
2,2,3068758,https://siva.jsstatic.com/id/28632/images/logo...,PT Home Credit Indonesia,Jakarta Barat (Jakarta Raya),Field Collection - Depok,Perbankan/Keuangan,1001,2000,Job Description:Mengunjungi Customer sesuai d...,23/09/19
3,3,3068757,https://siva.jsstatic.com/id/321/images/logo/3...,PT Repex Wahana,Palembang (Sumatera Selatan),Account Manager (Sales Executive)Palembang,Penjualan - Korporasi,1001,2000,\nKey Specific Responsibilities :\n \n\nIdent...,23/09/19
4,4,3068756,https://siva.jsstatic.com/id/28632/images/logo...,PT Home Credit Indonesia,Jakarta Selatan (Jakarta Raya),Field Collection - Bogor,Perbankan/Keuangan,1001,2000,Tugas dan Tanggung Jawab:Mengunjungi Customer...,23/09/19


In [5]:
features_column = "company_name,location,position,min_company_size,max_company_size,job_desc".split(',')
target_column = ["industry"]

In [6]:
df_features = df[features_column]
df_target = df[target_column]

In [7]:
df_features

,company_name,location,position,min_company_size,max_company_size,job_desc
0,PT Glassindo Utama,Jakarta Barat (Jakarta Raya),Estimator & Purchasing,51,200,Requirement : \n \n\nUmur 25 th s/d 40 th\nMi...
1,PT Home Credit Indonesia,Banten - Tangerang Area,Field Collection - Tangerang,1001,2000,Job Description:Mengunjungi Customer sesuai d...
2,PT Home Credit Indonesia,Jakarta Barat (Jakarta Raya),Field Collection - Depok,1001,2000,Job Description:Mengunjungi Customer sesuai d...
3,PT Repex Wahana,Palembang (Sumatera Selatan),Account Manager (Sales Executive)Palembang,1001,2000,\nKey Specific Responsibilities :\n \n\nIdent...
4,PT Home Credit Indonesia,Jakarta Selatan (Jakarta Raya),Field Collection - Bogor,1001,2000,Tugas dan Tanggung Jawab:Mengunjungi Customer...
5,PT Guntner Indonesia,Pasuruan (Jawa Timur) - Beji,Packing Engineer Staff,501,1000,Responsibilities:\n\nPrepare design packing\n...
6,PT Home Credit Indonesia,Jakarta Selatan (Jakarta Raya),Field Agent - Jakarta Selatan,1001,2000,Field Agent : Jakarta Selatan Halo Warga Jaka...
7,PT. SLS Bearindo,Surabaya (Jawa Timur),Salesman Surabaya,51,200,Responsibilities:\n\nResponsible for the sale...
8,PT Home Credit Indonesia,Jakarta Barat (Jakarta Raya),Field Agent - Jakarta Barat,1001,2000,Field Agent : Jakarta Barat Halo Warga Jakart...
9,PT Vale Indonesia Tbk,Jakarta Selatan (Jakarta Raya) - Sudirman,Corporate Tax Accountant,2001,5000,\nPurpose of the Role:\n\nPrepare and mainta...


In [17]:
URL = "https://www.jobstreet.co.id/id/job-search/job-vacancy.php?ojs=6"
avail = True
s = requests.session()
for i in range(1, 999999):
    pn = 1
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'}
    pay_load = {'key':'','area':1,'option':1,'pg':None,'classified':1,'src':16,'srcr':12}
    pay_load['pg'] = pn
    webpage = requests.get(URL, headers=headers, params=pay_load)
    soup = BeautifulSoup(webpage.text,'html.parser')
    links = soup.find_all('a',{'class':'position-title-link','data-job-id':re.compile(r'.*')})
    if not len(links):
        avail = False
    else:
        urls_logo, companies_name, locations, positions, industries, jobs_id, min_companies_size, max_companies_size, jobs_desc, date_posted = get_all_data(soup)
        df = pd.DataFrame.from_dict({"job_id":jobs_id,
                                    "url_logo":urls_logo,
                                    "company_name":companies_name,
                                    "location":locations,
                                    "position":positions,
                                    "industry":industries,
                                    "min_company_size":min_companies_size,
                                    "max_company_size":max_companies_size,
                                    "job_desc":jobs_desc,
                                    "posted":date_posted})

In [20]:
engine = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(user, password, host, dbname))
df.to_sql('jobstreet', engine, schema=None, if_exists='append', index=False)
# ??df.to_sql

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1045 (28000): Access denied for user 'umam'@'localhost' (using password: YES)
(Background on this error at: http://sqlalche.me/e/f405)

In [ ]:
url_logo = soup.find(id='job_ad_1').img["data-original"]
company_name = soup.find(id='job_ad_1').span.string
location = soup.find(id='job_ad_1').li["title"]
position = soup.find(id='job_ad_1').find('a',{'class':'position-title-link'})["data-job-title"]

title = soup.find(id='job_ad_1').find('a',{'class':'text-muted'})["title"]
industri = re.search(r'Lowongan (.*?) di', title).group(1)

company_size = enter company page
Job description = enter vacancy page
date = enter vacancy page
jobid = soup.find_all('a',{'class':'position-title-link','data-job-id':re.compile(r'.*')})[0]["data-job-id"]

Logo Perusahaan (simpan urlnya)
Nama Perusahaan
Lokasi Perusahaan
Posisi Pekerjaan (Job Title)
Industri
Ukuran Perusahaan
Job Description
Tanggal Loker (tanggal ditayangkan terletak di bawah laman detail job)


In [ ]:
page.headers

In [ ]:
position_links = []
pn = 1
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'}
pay_load = {'key':'','area':1,'option':1,'pg':None,'classified':1,'src':16,'srcr':12}
pay_load['pg'] = pn
r = requests.get(URL, headers=headers, params=pay_load)

# extract position <a> tags
soup = BeautifulSoup(r.text,'html.parser')
links = soup.find_all('a',{'class':'position-title-link','data-job-id':re.compile(r'.*')})

# if return nothing, means the function reach the last page, return results
if not len(links):
    loaded = False
else:
    position_links += links

In [ ]:
for jobid in range(100):
    a = soup.find(id='job_ad_{}'.format(jobid))
    if a == None and jobid > 0:
        print(jobid)
        break

In [ ]:
# title = soup.find(id='job_ad_1').find('a',{'class':'text-muted'})["title"]
# title = re.search(r'Lowongan (.*?) di', title).group(1)

# soup.find_all('div',{'class':'panel-body','id':re.compile(r'job_ad_(\d+)$')})[0].find('a', {'class':"company-name"}).span.string



In [ ]:
soup.find_all('div')

In [ ]:
re.search(r'Lowongan (.*?) di', title).group(1)

In [ ]:
for i, div in enumerate(soup.find_all('div')):
    if "HSEQ" in str(div):
        print(i)

In [ ]:
position_links[1]

In [ ]:
soup.find('div',{'id':'company_name'}).text.strip() if soup.find('div',{'id':'company_name'}) else None

In [ ]:
company_name

In [ ]:
import requests # for web requests
from bs4 import BeautifulSoup # a powerful HTML parser
import pandas as pd # for .csv file read and write
import re # for regular regression handling

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'}


""" get all position <a> tags for the list of job roles, results stored in a dictionary
<a> tag example:
<a class="position-title-link" id="position_title_3" href="https://www.jobstreet.com.sg/en/job/data-analyst-python-sas-sqlbank-35k-to-5k-gd-bonus5-days-west-6111488?fr=21"
target="_blank" title="View Job Details - Data Analyst (Python / SAS / SQL)(BANK / $3.5K to $5K + GD BONUS / 5 Days / West)" data-track="sol-job" data-job-id="6111488"
data-job-title="Data Analyst (Python / SAS / SQL)(BANK / $3.5K to $5K + GD BONUS / 5 Days / West)" data-type="organic" data-rank="3" data-page="1" data-posting-country="SG">
<h2 itemprop="title">Data Analyst (Python / SAS / SQL)(BANK / $3.5K to $5K + GD BONUS / 5 Days / West)</h2></a>"""
def linksByKeys(keys):
    ## keys: a list of job roles
    ## return: a dictionary of links

    links_dic = dict()
    # scrape key words one by one
    for key in keys:
        print('Scraping position: ', key, ' ...')
        links_dic[key] = linksByKey(key)
        print('{} {} positions found!'.format(len(links_dic[key]),key))
    return links_dic


""" get all position <a> tags for a single job role, triggered by linksByKeys function """
def linksByKey(key):
    ## key: a job role
    ## return: a list of links

    # parameters passed to  http get/post function
    base_url = 'https://www.jobstreet.com.sg/en/job-search/job-vacancy.php'
    pay_load = {'key':'','area':1,'option':1,'pg':None,'classified':1,'src':16,'srcr':12}
    pay_load['key'] = key

    # page number
    pn = 1

    position_links = []
    loaded = True
    while loaded:
        print('Loading page {} ...'.format(pn))
        pay_load['pg'] = pn
        r = requests.get(base_url, headers=headers, params=pay_load)

        # extract position <a> tags
        soup = BeautifulSoup(r.text,'html.parser')
        links = soup.find_all('a',{'class':'position-title-link','data-job-id':re.compile(r'.*')})

        # if return nothing, means the function reach the last page, return results
        if not len(links):
            loaded = False
        else:
            position_links += links
            pn += 1
    return position_links


""" parse HTML strings for the list of roles
<a> tag example:
<a class="position-title-link" id="position_title_3" href="https://www.jobstreet.com.sg/en/job/data-analyst-python-sas-sqlbank-35k-to-5k-gd-bonus5-days-west-6111488?fr=21"
target="_blank" title="View Job Details - Data Analyst (Python / SAS / SQL)(BANK / $3.5K to $5K + GD BONUS / 5 Days / West)" data-track="sol-job" data-job-id="6111488"
data-job-title="Data Analyst (Python / SAS / SQL)(BANK / $3.5K to $5K + GD BONUS / 5 Days / West)" data-type="organic" data-rank="3" data-page="1" data-posting-country="SG">"""
def parseLinks(links_dic):
    ## links_dic: a dictionary of links
    ## return: print parsed results to .csv file

    for key in links_dic:
        jobs = []
        for link in links_dic[key]:
            jobs.append([key] + parseLink(link))

        # transfrom the result to a pandas.DataFrame
        result = pd.DataFrame(jobs,columns=['key_word','job_id','job_title','country','job_link','company','company_region','company_industry','company_size','experence_requirement','working_location','description'])

        # add a column denoting if the position is posted by a recuriter company
        result['postedByHR'] = result.company_industry.apply(lambda x:True if x and x.find('Human Resources')>-1 else False)

        # save result,
        file_name = key+'.csv'
        result.to_csv(file_name,index=False)


""" parse a single <a> tag, extract the information, triggered by parseLinks function """
def parseLink(link):
	## link: a single position <a> tag
	## return: information of a single position

	# unique id assigned to a position
	job_id = link['data-job-id'].strip()
	# job title
	job_title = link['data-job-title'].strip()
	# posted country
	country = link['data-posting-country'].strip()
	# the web address towards to the post detail page
	job_href = link['href']
	# go to post detail page, and fetch information
	other_detail = getJobDetail(job_href)
	return [job_id,job_title,country,job_href] + other_detail


""" extract details from post detail page """
def getJobDetail(job_href):
    ## job_href: a post url
    ## retun: post details from the detail page

    print('Scraping ',job_href,'...')
    r = requests.get(job_href)
    soup = BeautifulSoup(r.text,'html.parser')
    # company who posts the position, very often is a recuriter company
    company_name = soup.find('div',{'id':'company_name'}).text.strip() if soup.find('div',{'id':'company_name'}) else None
    # years of working experience required
    years_of_experience= soup.find('span',{'id':'years_of_experience'}).text.strip() if soup.find('span',{'id':'years_of_experience'}) else None
    # location of the company
    company_location = soup.find('span',{'id':'single_work_location'}).text.strip() if soup.find('span',{'id':'single_work_location'}) else None
    # industry of the company who posts the position, very often is a recuriter company
    company_industry = soup.find('p',{'id':'company_industry'}).text.strip() if soup.find('p',{'id':'company_industry'}) else None
    # size of the company who posts the position
    company_size = soup.find('p',{'id':'company_size'}).text.strip() if soup.find('p',{'id':'company_size'}) else None
    # working location
    job_location = soup.find('p',{'id':'address'}).text.strip() if soup.find('p',{'id':'address'}) else None
    # job description, which contains information about job scope, requirements and sometimes a brief introduction about the comapny
    job_description = soup.find('div',{'id':'job_description'}).text.strip() if soup.find('div',{'id':'job_description'}) else None
    return [company_name,company_location,company_industry,company_size,years_of_experience,job_location,job_description]

def main():

    # a list of job roles to be crawled
    key_words = ['data scientist']
    s = requests.session()
    links_dic = linksByKeys(key_words)
    parseLinks(links_dic)

if __name__ == '__main__':
	main()
